Imports

In [ ]:
import perceval as pcvl
import os
import numpy as np
from perceval.components.unitary_components import BS, PERM, PS
from pprint import pprint
import tqdm
import time

We want to reproduce this circuit:

![](chsh.png)

Define input state and circuit:

In [ ]:
input_state = pcvl.BasicState()

In [ ]:
circuit = pcvl.Circuit()

The circuit starts with two 50/50 beam-splitters:

In [ ]:
circuit.add(0, BS())
circuit.add(2, BS())

In [ ]:
pcvl.pdisplay(circuit)

Then we have a permutation of modes. PERM is a simple perceval component:

In [ ]:
perm_example = PERM([2,0,1])
pcvl.pdisplay(perm_example)

Here we switch Alice's second mode with Bob's first mode:

In [ ]:
circuit.add(0, PERM())

How does it look so far?

In [ ]:
pcvl.pdisplay(circuit)

Now let's add the two interferometers that define the measurement bases. We keep the angles as parameters to be set during the Bell test.

In [ ]:
psi_A = pcvl.P('psi_A')
psi_B = pcvl.P('psi_B')

circuit.add(0, PS(psi_A))
circuit.add(3, PS(psi_B))

In [ ]:
circuit.add(0, BS())
circuit.add(2, BS())

In [ ]:
phi_A = pcvl.P('phi_A')
phi_B = pcvl.P('phi_B')

circuit.add(0, PS(phi_A))
circuit.add(3, PS(phi_B))

In [ ]:
circuit.add(0, BS())
circuit.add(2, BS())

How does it look now?

In [ ]:
pcvl.pdisplay(circuit)

What are the angles needed for the CHSH measurements?

![](chsh_meas.png)

This means we can just set the psi angles to 0:

In [ ]:
psi_A.set_value(0)
psi_B.set_value(0)

Let us define the processor and sampler for our simulation:

In [ ]:
processor = pcvl.Processor("SLOS", circuit)

We have a post-selection rule in our dual rail encoding: there should be 1 photon in Alice's mode and 1 photon in Bob's modes:

In [ ]:
post_selection_rule = pcvl.PostSelect("[0,1] == 1 & [2,3] == 1")
processor.set_postselection(post_selection_rule)

In [ ]:
processor.with_input(input_state)

In [ ]:
sampler = pcvl.algorithm.Sampler(processor)

And now let us run a sampling simulation for all the inputs of the Bell test:

In [ ]:
total_samples = 10000

In [ ]:
for x in [-np.pi/2, 0]:
    for y in [-np.pi/4, np.pi/4]:
        phi_A.set_value(x)
        phi_B.set_value(y)
        sample_count = sampler.sample_count(total_samples/4)
        print(sample_count['results'])

Those are just the raw results. Now we have to compute the CHSH expression.

Recall: $B_{CHSH} = |<A_1B_1> + <A_1B_2> + <A_2B_1> - <A_2B_2>|$

With $<A_xB_y> = p(00|xy) - p(01|xy) - p(10|xy) + p(11|xy)$

In [ ]:
correlator_list = []

for x in [0, -np.pi/2]:
    for y in [-np.pi/4, np.pi/4]:
        phi_A.set_value(x)
        phi_B.set_value(y)
        sample_count = sampler.sample_count(total_samples/4)
        
        correlator = (sample_count['results'][pcvl.BasicState('|1,0,1,0>')] 
                      - sample_count['results'][pcvl.BasicState('|1,0,0,1>')]
                      - sample_count['results'][pcvl.BasicState('|0,1,1,0>')]
                      + sample_count['results'][pcvl.BasicState('|0,1,0,1>')])

        correlator = correlator/(total_samples/4)
        
        correlator_list.append(correlator)

In [ ]:
correlator_list

In [ ]:
CHSH = np.abs(correlator_list[0] + correlator_list[1] + correlator_list[2] - correlator_list[3])
CHSH

In [ ]:
2*np.sqrt(2)

Now let's try it on the cloud: use token that you can create in the web interface

In [ ]:
token_cloud = ''

Define remote processor instead of usual processor:

In [ ]:
remote_simulator = pcvl.RemoteProcessor("sim:ascella", token_cloud)

You can easily access the specificities of the machines on the cloud:

In [ ]:
specs = remote_simulator.specs
pcvl.pdisplay(specs["specific_circuit"])

In [ ]:
print("Platform constraints:")
pprint(specs["constraints"])
print("\nPlatform supported parameters:")
pprint(specs["parameters"])

Set the circuit as usual:

In [ ]:
remote_simulator.set_circuit(circuit)
remote_simulator.with_input(input_state)

In [ ]:
# optional if you want to change the noise parameters in a simulator
#remote_simulator.set_parameters({  
#    "HOM": .95,
#    "transmittance": .1,
#    "g2": .01
#})

Post selection and photon click rules:

In [ ]:
remote_simulator.min_detected_photons_filter(1)

In [ ]:
remote_simulator.set_postselection(post_selection_rule)

Define remote job:

In [ ]:
nsamples = 200000
sampler = pcvl.algorithm.Sampler(remote_simulator, max_shots_per_call=nsamples) 

sampler.default_job_name = "My sampling job"  

remote_job = sampler.sample_count.execute_async(nsamples)

In [ ]:
while not remote_job.is_complete:
    time.sleep(0.02)

print(f"Job status = {remote_job.status()}")

Get results:

In [ ]:
results = remote_job.get_results()
print(results['results'])

Now let's do it for the 4 correlators in the loop:

In [ ]:
correlator_list = []

for x in [0, -np.pi/2]:
    for y in [-np.pi/4, np.pi/4]:
        phi_A.set_value(x)
        phi_B.set_value(y)

        remote_job = sampler.sample_count.execute_async(nsamples)  
        while not remote_job.is_complete:
            time.sleep(0.02)
        print(f"Job status = {remote_job.status()}")

        results = remote_job.get_results()

        total_counts = (results['results'][pcvl.BasicState('|1,0,1,0>')] 
                      + results['results'][pcvl.BasicState('|1,0,0,1>')]
                      + results['results'][pcvl.BasicState('|0,1,1,0>')]
                      + results['results'][pcvl.BasicState('|0,1,0,1>')])
        
        correlator = (results['results'][pcvl.BasicState('|1,0,1,0>')] 
                      - results['results'][pcvl.BasicState('|1,0,0,1>')]
                      - results['results'][pcvl.BasicState('|0,1,1,0>')]
                      + results['results'][pcvl.BasicState('|0,1,0,1>')])

        prob_list = [results['results'][pcvl.BasicState('|1,0,1,0>')]/total_counts,
                     results['results'][pcvl.BasicState('|1,0,0,1>')]/total_counts,
                     results['results'][pcvl.BasicState('|0,1,1,0>')]/total_counts,
                     results['results'][pcvl.BasicState('|0,1,0,1>')]/total_counts]

        correlator = correlator/total_counts
        
        correlator_list.append(correlator)

Check the correlators and check that the probabilities sum to 1:

In [ ]:
correlator_list

In [ ]:
sum(prob_list)

We get a CHSH value which should be less than the max violation because of the noise:

In [ ]:
CHSH = np.abs(correlator_list[0] + correlator_list[1] + correlator_list[2] - correlator_list[3])
CHSH